In [81]:
import os
result_folder = '/2parameterSA_volume_area'
test = os.listdir('C:/Users/20181270/OneDrive - TU Eindhoven/PhD/TRNSYS/Publication1/with_summer_loop'+result_folder)
test

['base_PV_V0_15_A10_control_signal.txt',
 'base_PV_V0_15_A10_energy.txt',
 'base_PV_V0_15_A10_temp_flow.txt',
 'base_PV_V0_15_A12_control_signal.txt',
 'base_PV_V0_15_A12_energy.txt',
 'base_PV_V0_15_A12_temp_flow.txt',
 'base_PV_V0_15_A14_control_signal.txt',
 'base_PV_V0_15_A14_energy.txt',
 'base_PV_V0_15_A14_temp_flow.txt',
 'base_PV_V0_15_A16_control_signal.txt',
 'base_PV_V0_15_A16_energy.txt',
 'base_PV_V0_15_A16_temp_flow.txt',
 'base_PV_V0_15_A18_control_signal.txt',
 'base_PV_V0_15_A18_energy.txt',
 'base_PV_V0_15_A18_temp_flow.txt',
 'base_PV_V0_15_A20_control_signal.txt',
 'base_PV_V0_15_A20_energy.txt',
 'base_PV_V0_15_A20_temp_flow.txt',
 'base_PV_V0_1_A16_control_signal.txt',
 'base_PV_V0_1_A16_energy.txt',
 'base_PV_V0_1_A16_temp_flow.txt',
 'base_PV_V0_1_A18_control_signal.txt',
 'base_PV_V0_1_A18_energy.txt',
 'base_PV_V0_1_A18_temp_flow.txt',
 'base_PV_V0_25_A10_control_signal.txt',
 'base_PV_V0_25_A10_energy.txt',
 'base_PV_V0_25_A10_temp_flow.txt',
 'base_PV_V0_25_

In [82]:
labels = []
for i in test:
    if '_temp_flow' in i:
        prefix = i[:-14]
    elif '_control_signal' in i:
        prefix = i[:-19]
    elif '_energy' in i:
        prefix = i[:-11]
    labels.append(prefix)
        
labels

['base_PV_V0_15_A10',
 'base_PV_V0_15_A10',
 'base_PV_V0_15_A10',
 'base_PV_V0_15_A12',
 'base_PV_V0_15_A12',
 'base_PV_V0_15_A12',
 'base_PV_V0_15_A14',
 'base_PV_V0_15_A14',
 'base_PV_V0_15_A14',
 'base_PV_V0_15_A16',
 'base_PV_V0_15_A16',
 'base_PV_V0_15_A16',
 'base_PV_V0_15_A18',
 'base_PV_V0_15_A18',
 'base_PV_V0_15_A18',
 'base_PV_V0_15_A20',
 'base_PV_V0_15_A20',
 'base_PV_V0_15_A20',
 'base_PV_V0_1_A16',
 'base_PV_V0_1_A16',
 'base_PV_V0_1_A16',
 'base_PV_V0_1_A18',
 'base_PV_V0_1_A18',
 'base_PV_V0_1_A18',
 'base_PV_V0_25_A10',
 'base_PV_V0_25_A10',
 'base_PV_V0_25_A10',
 'base_PV_V0_25_A12',
 'base_PV_V0_25_A12',
 'base_PV_V0_25_A12',
 'base_PV_V0_25_A14',
 'base_PV_V0_25_A14',
 'base_PV_V0_25_A14',
 'base_PV_V0_25_A16',
 'base_PV_V0_25_A16',
 'base_PV_V0_25_A16',
 'base_PV_V0_25_A18',
 'base_PV_V0_25_A18',
 'base_PV_V0_25_A18',
 'base_PV_V0_25_A20',
 'base_PV_V0_25_A20',
 'base_PV_V0_25_A20',
 'base_PV_V0_2_A10',
 'base_PV_V0_2_A10',
 'base_PV_V0_2_A10',
 'base_PV_V0_2_A12'

In [83]:
import numpy as np
labels2 = np.array(labels)

In [84]:
x = np.unique(labels2)
x

array(['PVT_Batt_6_V0_15_A10', 'PVT_Batt_6_V0_15_A12',
       'PVT_Batt_6_V0_15_A14', 'PVT_Batt_6_V0_15_A16',
       'PVT_Batt_6_V0_15_A18', 'PVT_Batt_6_V0_1_A12',
       'PVT_Batt_6_V0_1_A14', 'PVT_Batt_6_V0_1_A16',
       'PVT_Batt_6_V0_1_A18', 'PVT_Batt_6_V0_25_A10',
       'PVT_Batt_6_V0_25_A12', 'PVT_Batt_6_V0_25_A14',
       'PVT_Batt_6_V0_25_A16', 'PVT_Batt_6_V0_25_A18',
       'PVT_Batt_6_V0_25_A20', 'PVT_Batt_6_V0_2_A12',
       'PVT_Batt_6_V0_2_A14', 'PVT_Batt_6_V0_2_A16',
       'PVT_Batt_6_V0_3_A10', 'PVT_Batt_6_V0_3_A12',
       'PVT_Batt_6_V0_3_A14', 'PVT_Batt_6_V0_3_A16',
       'PVT_Batt_6_V0_3_A18', 'PVT_Batt_9_V0_15_A16',
       'PVT_Batt_9_V0_15_A18', 'PVT_Batt_9_V0_15_A20',
       'PVT_Batt_9_V0_1_A12', 'PVT_Batt_9_V0_25_A10',
       'PVT_Batt_9_V0_25_A12', 'PVT_Batt_9_V0_25_A18',
       'PVT_Batt_9_V0_2_A12', 'PVT_Batt_9_V0_2_A16',
       'PVT_Batt_9_V0_2_A18', 'PVT_Batt_9_V0_2_A20',
       'PVT_Batt_9_V0_3_A16', 'PVT_Batt_9_V0_3_A18', 'PVT_V0_15_A10',
       'PVT_

In [85]:
len(x)

117

In [86]:
import pandas as pd
dfsobol = pd.read_csv('Sobol_samples_2parameterSA_volume_area.csv')
dfsobol = dfsobol.drop_duplicates()
dfsobol.index = np.arange(len(dfsobol))

In [87]:
sobol_labels = []
for i in range(len(dfsobol)):
    sobol_labels.append(dfsobol['design_case'][i]+'_V'+str(dfsobol['volume'][i]).replace('.','_')+'_A'+str(dfsobol['coll_area'][i]))
sobol_labels = np.array(sobol_labels)
sobol_labels

array(['PVT_Batt_6_V0_1_A12', 'PVT_Batt_6_V0_2_A12',
       'PVT_Batt_9_V0_1_A12', 'PVT_Batt_9_V0_2_A12', 'base_PV_V0_2_A18',
       'base_PV_V0_1_A18', 'base_PV_V0_2_A16', 'ST_V0_2_A18',
       'ST_V0_2_A16', 'ST_V0_1_A18', 'base_PV_V0_1_A16', 'ST_V0_1_A16',
       'PVT_Batt_9_V0_25_A10', 'PVT_Batt_9_V0_25_A18',
       'base_PV_V0_25_A10', 'base_PV_V0_25_A18', 'ST_V0_15_A16',
       'ST_V0_15_A14', 'PVT_V0_15_A16', 'PVT_V0_15_A14', 'base_V0_2_A12',
       'PVT_V0_3_A16', 'PVT_V0_3_A18', 'base_PV_V0_3_A16',
       'base_PV_V0_3_A18', 'ST_V0_25_A14', 'ST_V0_25_A16',
       'base_V0_25_A14', 'base_V0_25_A16', 'base_V0_15_A14',
       'base_V0_15_A16', 'PVT_Batt_6_V0_15_A18', 'PVT_Batt_6_V0_25_A18',
       'PVT_Batt_6_V0_15_A10', 'PVT_V0_15_A18', 'PVT_V0_15_A10',
       'PVT_V0_25_A18', 'PVT_Batt_6_V0_25_A10', 'PVT_V0_25_A10',
       'base_PV_V0_15_A10', 'base_PV_V0_3_A10', 'base_PV_V0_15_A18',
       'PVT_Batt_6_V0_3_A10', 'PVT_Batt_6_V0_3_A18',
       'PVT_Batt_6_V0_25_A16', 'PVT_Batt_6

In [88]:
result = np.concatenate((labels2, sobol_labels))

In [89]:
# missing samples from simulation results
set(sobol_labels) - set(labels2)

{'ST_V0_15_A12', 'ST_V0_2_A10'}